In [2]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import re
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

driver = uc.Chrome(version_main=118)

html_file = Path.cwd() / 'Underdog Fantasy_ Results.mhtml'
driver.get(html_file.as_uri())

In [13]:
leagues = {
    'DET': ['NFL'],
    'KC': ['NFL'],
    'CIN': ['NFL', 'NCAA'],
    'CLE': ['NFL', 'NCAA'],
    'TB': ['NFL'],
    'MIN': ['NFL', 'NCAA'],
    'HOU': ['NFL'],
    'BAL': ['NFL'],
    'SF': ['NFL'],
    'PIT': ['NFL', 'NCAA'],
    'JAX': ['NFL'],
    'IND': ['NFL'],
    'CAR': ['NFL'],
    'ATL': ['NFL'],
    'WAS': ['NFL', 'NCAA'],
    'TEN': ['NFL', 'NCAA'],
    'NO': ['NFL'],
    'MIA': ['NFL', 'NCAA'],
    'LAC': ['NFL'],
    'LV': ['NFL'],
    'DEN': ['NFL'],
    'PHI': ['NFL'],
    'NE': ['NFL'],
    'GB': ['NFL'],
    'CHI': ['NFL'],
    'LAR': ['NFL'],
    'SEA': ['NFL'],
    'DAL': ['NFL'],
    'NYG': ['NFL'],
    'BUF': ['NFL'],
    'NYJ': ['NFL'],
    'ARI': ['NFL', 'NCAA'],
    'NCST': ['NCAA'],
    'UCONN': ['NCAA'],
    'NEB': ['NCAA'],
    'FLA': ['NCAA'],
    'UTH': ['NCAA'],
    'BC': ['CFL'],
    'CGY': ['CFL'],
    'EDM': ['CFL'],
    'HAM': ['CFL'],
    'WPG': ['CFL'],
    'MTL': ['CFL'],
    'SAS': ['CFL'],
    'TOR': ['CFL'],
    'UMASS': ['NCAA'],
    'NMS': ['NCAA'],
    'SDSU': ['NCAA'],
    'OHI': ['NCAA'],
    'VAN': ['NCAA'],
    'HAW': ['NCAA'],
    'SJS': ['NCAA'],
    'USC': ['NCAA'],
    'OTT': ['CFL'],
    'KNT': ['NCAA'],
    'UCF': ['NCAA'],
    'CMC': ['NCAA'],
    'MSU': ['NCAA'],
    'LOU': ['NCAA'],
    'GT': ['NCAA'],
    'TCU': ['NCAA'],
    'COL': ['NCAA'],
    'UTS': ['NCAA'],
    'IOW': ['NCAA'],
    'STA': ['NCAA'],
    'DUK': ['NCAA'],
    'FSU': ['NCAA'],
    'LSU': ['NCAA'],
    'ORS': ['NCAA'],
    'ILL': ['NCAA'],
    'TOL': ['NCAA'],
    'BOISE': ['NCAA'],
    'WIS': ['NCAA'],
    'WST': ['NCAA'],
    'MSST': ['NCAA'],
    'TT': ['NCAA'],
    'ORE': ['NCAA'],
    'BAMA': ['NCAA'],
    'TEX': ['NCAA'],
    'MIS': ['NCAA'],
    'TUL': ['NCAA'],
    'ISU': ['NCAA'],
    'MFL': ['NCAA'],
    'TXAM': ['NCAA'],
    'PUR': ['NCAA'],
    'VT': ['NCAA'],
    'ND': ['NCAA'],
    'WF': ['NCAA'],
    'TEP': ['NCAA'],
    'CSU': ['NCAA'],
    'ARK': ['NCAA'],
    'BYU': ['NCAA'],
    'WVU': ['NCAA'],
    'WKY': ['NCAA'],
    'OSU': ['NCAA'],
    'NW': ['NCAA'],
    'NC': ['NCAA'],
    'UGA': ['NCAA'],
    'SC': ['NCAA'],
    'GSO': ['NCAA'],
    'IU': ['NCAA'],
    'ODU': ['NCAA'],
    'PSU': ['NCAA'],
    'KST': ['NCAA'],
    'MIZ': ['NCAA'],
    'MEM': ['NCAA'],
    'MAR': ['NCAA'],
    'MICH': ['NCAA'],
    'RUT': ['NCAA'],
    'SMU': ['NCAA'],
    'ASU': ['NCAA'],
    'VAN': ['NCAA'],
    'KEN': ['NCAA'],
    'OKL': ['NCAA'],
    'AUB': ['NCAA'],
    'UVA': ['NCAA'],
    'CC': ['NCAA'],
    'GST': ['NCAA'],
    'OKS': ['NCAA'],
    'LT': ['NCAA'],
    'KAN': ['NCAA'],
    'SYR': ['NCAA'],
    'MTS': ['NCAA']
}

def extract_team_codes(text, pattern):
    match = re.search(pattern, text)
    if match:
        return [match.group(1), match.group(3)]
    return []

# All Bets Exporter (Updated 10/18/23)

In [14]:
betcontainer = driver.find_element(By.CLASS_NAME, 'styles__resultsPickEmPage__hZRXU')
betlist = betcontainer.find_elements(By.CSS_SELECTOR, 'div[data-entry-slip-id]')

data = []

for bet in betlist:
    try:
        players = bet.find_element(By.CLASS_NAME, 'styles__playerNames__DTWWj').text
        matchPattern = r'([A-Z]+) \d+ (vs|@) ([A-Z]+) \d+'
        matchInfoRaw = bet.find_element(By.CLASS_NAME, 'styles__matchInfoText__ZOV5w').text
        teamsList = extract_team_codes(matchInfoRaw, matchPattern)

    except:
        continue

    if len(teamsList) == 0:
        print(f'{players} Teams Not Found')
        continue
    
    if teamsList[0] in leagues and teamsList[1] in leagues:
        team1Leagues = leagues[teamsList[0]]
        team2Leagues = leagues[teamsList[1]]
        
        matches = list(set(team1Leagues) & set(team2Leagues))
        
        if len(matches) == 1:
            league = matches[0]
            
        else:
            print(f'{teamsList}: League mismatch or multple matches found')
            league = 'Unknown'
    
    else:
        print(f'{teamsList}: at least one of these teams are not in Leagues Dictionary')
        league = 'Unknown'
    
#     Un-Comment for CS:GO Only
#     if 'CS:GO' not in bet.text and 'Val' not in bet.text:
#         players = bet.find_element(By.CLASS_NAME, 'styles__playerNames__DTWWj').text
#         print(f'{players}: Not CS:GO Nor VAL')
#         continue

########### LOOPING THROUGH PLAYERS #################

    playerWrapper = bet.find_element(By.CLASS_NAME, 'styles__playerCellWrapper__jOm74')
    playerChoiceList = playerWrapper.find_elements(By.CLASS_NAME, 'styles__choiceText__qT7Br')
    
    lowCount = 0
    highCount = 0
    
    for choice in playerChoiceList:
        if choice.text == 'Lower':
            lowCount += 1
        else:
            highCount += 1
        
    if lowCount > highCount:
        betType = 'Lower'
    else:
        betType = 'Higher'
        
    date = re.findall(r'(\d+/\d+/\d+)', matchInfoRaw)
    if len(date) == 0:
        date = datetime.now().strftime('%m/%d/%Y')
    else:
        date = date[0]
    
    title = bet.find_element(By.CLASS_NAME, 'styles__slipTitle__Rcwjr').text
    
    if '5 Picks' in title:
        picks = 5
        betsize = int(title.split('$')[-1].replace(',', '')) / 20
    elif '4 Picks' in title:
        picks = 4
        betsize = int(title.split('$')[-1].replace(',', '')) / 10
    elif '3 Picks' in title:
        picks = 3
        betsize = int(title.split('$')[-1].replace(',', '')) / 6
    elif '2 Picks' in title:
        picks = 2
        betsize = int(title.split('$')[-1].replace(',', '')) / 3
        
    bottomRowText = bet.find_element(By.CLASS_NAME, 'styles__bottomRow__kcxwS').text.split()
    
    if bottomRowText[0] == 'Lost':
        outcome = 'Loss'
        profit = 0 - betsize
    elif bottomRowText[0] == 'Won':
        outcome = 'Win'
        profit = int(bottomRowText[-1].replace('$', '').replace(',', '')) - betsize
    else:
        print(f'{players} on {date} unknown outcome: {bottomRowText}. outcome set to unknown and profit to 0.')
        outcome = 'Unknown'
        profit = 0
        
    new_row = {
        'League': league,
        'Players': players,
        'Bet Type': betType,
        'Picks': picks,
        'Date': date,
        'Bet Size': betsize,
        'Outcome': outcome,
        'Profit': profit
    }
    
    data.append(new_row)
    
df = pd.DataFrame(data) 

['CIN', 'ARI']: League mismatch or multple matches found
['CIN', 'TEN']: League mismatch or multple matches found
['CIN', 'TEN']: League mismatch or multple matches found
['WAS', 'ARI']: League mismatch or multple matches found
['WAS', 'ARI']: League mismatch or multple matches found


In [16]:
df.to_excel('UnderdogFantasyBets.xlsx', index=False)
df.head(15)

,League,Players,Bet Type,Picks,Date,Bet Size,Outcome,Profit
0,NFL,"Allen, Prescott, Lamb, Palmer, Herbert",Lower,5,10/16/23,100.0,Loss,-100.0
1,NFL,"Taylor, Diggs, Davis, Waller, Allen",Lower,5,10/15/23,100.0,Loss,-100.0
2,NFL,"Kupp, Stafford, Dobbs, Brown, Nacua",Lower,5,10/15/23,100.0,Loss,-100.0
3,NFL,"Goff, Mayfield, St. Brown, Godwin, Evans",Lower,5,10/15/23,100.0,Loss,-100.0
4,NFL,"Bourne, Garoppolo, Meyers, Adams, Jones",Lower,5,10/15/23,100.0,Loss,-100.0
5,NFL,"Brown, Smith, Hurts, Wilson, Wilson",Lower,5,10/15/23,100.0,Loss,-100.0
6,NFL,"Walker, Cooper, Purdy, Samuel, Aiyuk",Lower,5,10/15/23,100.0,Loss,-100.0
7,NFL,"Hockenson, Moore, Fields, Addison, Cousins",Lower,5,10/15/23,100.0,Win,1900.0
8,NFL,"Pittman Jr., Lawrence, Kirk, Ridley, Minshew II",Lower,5,10/15/23,100.0,Loss,-100.0
9,NFL,"Higgins, Chase, Metcalf, Smith, Burrow",Lower,5,10/15/23,100.0,Loss,-100.0


# Football Correlations (Outdated)

In [4]:
betcontainer = driver.find_element(By.CLASS_NAME, 'styles__resultsPickEmPage__hZRXU')
betlist = betcontainer.find_elements(By.CSS_SELECTOR, 'div[data-entry-slip-id]')

data = []

for bet in betlist:
    if 'Lower' in bet.text:
        pass
    elif 'Higher' in bet.text:
        pass
    else:
        continue 
    
    inforaw = bet.find_element(By.CLASS_NAME, 'styles__matchInfoText__ZOV5w').text.split('-')[0]
#     for item in inforaw:
#         if any(char.isdigit() for char in item):
#             inforaw.remove(item)
#     gameinfo = f'{inforaw[0]} {inforaw[1]} {inforaw[2]}'
    
    playercontainer = bet.find_element(By.CLASS_NAME, 'styles__playerCellWrapper__jOm74')
    playercards = playercontainer.find_elements(By.CSS_SELECTOR, 'div[data-appearance-id]')
    
    for player in playercards:
        game = {}
        projraw = player.find_element(By.CLASS_NAME, 'styles__statLine__QGxD2').text
        
        if 'Receiving' in projraw:
            league = 'NFL'
            position = 'WR'
            
        elif 'Passing' in projraw:
            league = 'NFL'
            position = 'QB'
            
        elif 'Rec Yards' in projraw:
            league = 'CFB'
            position = 'WR'
        
        elif 'Pass Yards' in projraw:
            league = 'CFB'
            position = 'QB'
            
        else:
            print(f'{projraw}, not part of the strategy')
            continue
            
        name = player.find_element(By.CLASS_NAME, 'styles__playerName__tRwnv').text.replace('(2H)', '')
        team = player.find_element(By.CLASS_NAME, 'styles__matchInfoText__ZOV5w').text.split()[0]
        projsplit = projraw.split('-')[-1]
        proj = projsplit.split()[0]
        try:
            actual_stat = player.find_element(By.CLASS_NAME, 'styles__resultStat__tHnnc').text.split()[0]
        except:
            print(f'{inforaw}: player push')
            actual_stat = 0
            
        if '(2H)' in bet.text:
            game['Game'] = inforaw + ' (2H)'
            
        else:
            game['Game'] = inforaw
            
        game['League'] = league
        game['Position'] = position
        game['Name'] = name
        game['Team'] = team
        game['Projection'] = float(proj)
        game['Actual'] = float(actual_stat)
        
        data.append(game)
        
        
        
        

Lower  - 0.5 Shots Attempted, not part of the strategy
DEN 9 @ BAL 10 : player push
DEN 9 @ BAL 10 : player push
Lower  - 0.5 Shots Attempted, not part of the strategy
Lower  - 254.5 Total Yards, not part of the strategy


In [5]:
df = pd.DataFrame(data)
df1 = df.drop_duplicates(subset=df.columns.difference(['Game']))
df1.to_excel('UnderdogFantasyCorrelations.xlsx', index=False)
df1

,Game,League,Position,Name,Team,Projection,Actual
0,NO 16 @ TB 17 (2H),NFL,WR,Chris Olave,NO,22.5,55.0
1,NO 16 @ TB 17 (2H),NFL,QB,Tom Brady,TB,157.5,177.0
2,NO 16 @ TB 17 (2H),NFL,WR,Mike Evans,TB,32.5,45.0
3,NO 16 @ TB 17 (2H),NFL,WR,Chris Godwin,TB,36.5,37.0
4,NO 16 @ TB 17 (2H),NFL,QB,Andy Dalton,NO,94.5,87.0
...,...,...,...,...,...,...,...
644,BUF 24 @ NE 10 (2H),NFL,QB,Mac Jones,NE,131.5,97.0
645,BUF 24 @ NE 10 (2H),NFL,WR,DeVante Parker,NE,20.5,9.0
646,BUF 24 @ NE 10 (2H),NFL,QB,Josh Allen,BUF,132.5,105.0
649,NE 10 vs BUF 24,NFL,WR,DeVante Parker,NE,35.5,16.0


# CSGO Correlations (Outdated)

In [3]:
betcontainer = driver.find_element(By.CLASS_NAME, 'styles__resultsPickEmPage__hZRXU')
betlist = betcontainer.find_elements(By.CSS_SELECTOR, 'div[data-entry-slip-id]')

data = []

for bet in betlist:
    if 'Lower' in bet.text:
        pass
    elif 'Higher' in bet.text:
        pass
    else:
        continue 
    
    inforaw = bet.find_element(By.CLASS_NAME, 'styles__matchInfoText__ZOV5w').text.split('-')[0]
    inforaw = inforaw.replace(' 0 ', ' ')
    inforaw = inforaw.split()
    gameinfo = f'{inforaw[0]} {inforaw[1]} {inforaw[2]}'
    
    playercontainer = bet.find_element(By.CLASS_NAME, 'styles__playerCellWrapper__jOm74')
    playercards = playercontainer.find_elements(By.CSS_SELECTOR, 'div[data-appearance-id]')
    
    for player in playercards:
        game = {}
        projraw = player.find_element(By.CLASS_NAME, 'styles__statLine__QGxD2').text
        
        if 'Kills' in projraw:
            stat = 'Kills'
            
        elif 'Headshots' in projraw:
            stat = 'Headshots'
            
        else:
            print(f'{projraw}, not part of the strategy')
            continue
            
        name = player.find_element(By.CLASS_NAME, 'styles__playerName__tRwnv').text.replace('CS:GO', '')
        team = player.find_element(By.CLASS_NAME, 'styles__matchInfoText__ZOV5w').text.split()[0]
        projsplit = projraw.split('-')[-1]
        proj = projsplit.split()[0]
        try:
            actual_stat = player.find_element(By.CLASS_NAME, 'styles__resultStat__tHnnc').text.split()[0]
        except:
            print(f'{gameinfo}: player push')
            actual_stat = 0
            
        game['Game'] = gameinfo    
        game['Stat'] = stat
        game['Name'] = name
        game['Team'] = team
        game['Projection'] = float(proj)
        game['Actual'] = float(actual_stat)
        
        data.append(game)

RGN @ FLYT: player push
FLYT vs RGN: player push
yNiP vs HAVU: player push
yNiP vs HAVU: player push
Higher  - 3.5 Clearances, not part of the strategy
Higher  - 74.5 Passes, not part of the strategy


In [4]:
df = pd.DataFrame(data)
df1 = df.drop_duplicates(subset=df.columns.difference(['Game']))
df1.to_excel('Underdog.CSGO.Correlations.xlsx', index=False)
df1

,Game,Stat,Name,Team,Projection,Actual
0,PAQ vs ODDK,Kills,ALLE,PAQ,34.0,30.0
1,PAQ vs ODDK,Kills,nython,PAQ,36.0,43.0
2,PAQ vs ODDK,Kills,remix,ODDK,34.0,44.0
3,PAQ vs ODDK,Kills,naitte,ODDK,32.0,37.0
8,FUSC @ FLMG,Kills,Lcm,FUSC,30.5,28.0
...,...,...,...,...,...,...
2695,ERUP @ IHC,Kills,dobu,ERUP,36.0,26.0
2700,WIND @ MIBa,Kills,bichop,WIND,35.0,32.0
2701,WIND @ MIBa,Kills,card,MIBa,34.5,19.0
2702,WIND @ MIBa,Kills,phx,MIBa,32.0,25.0
